In [57]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [58]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IRP

In [59]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [60]:
import requests
import json

def get_balance(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8434R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "AFHR_FLPR_YN": "N",  # 시간외단일가여부
        "OFL_YN": "",  # 공란
        "INQR_DVSN": "01",  # 조회구분
        "UNPR_DVSN": "01",  # 단가구분
        "FUND_STTL_ICLD_YN": "N",  # 펀드결제분포함여부
        "FNCG_AMT_AUTO_RDPT_YN": "N",  # 융자금액자동상환여부        
        "PRCS_DVSN": "00",  # 처리구분(00: 전일매매포함)
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()
    

In [61]:
def get_balance_irp(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/pension/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC2208R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "ACCA_DVSN_CD":"00",
        "INQR_DVSN":"00",
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()

In [62]:
json_0 = get_balance(token_0, api_key_0, api_secret_0, acc_no_0)
json_1 = get_balance(token_1, api_key_1, api_secret_1, acc_no_1)
json_2 = get_balance(token_2, api_key_2, api_secret_2, acc_no_2)
json_3 = get_balance(token_3, api_key_isa, api_secret_isa, acc_no_isa)
json_4 = get_balance_irp(token_4, api_key_irp, api_secret_irp, acc_no_irp)
json_4

{'ctx_area_fk100': '64012548^29^00^00^                                                                                  ',
 'ctx_area_nk100': '                                                                                                    ',
 'output1': [{'cblc_dvsn_name': '사용자',
   'prdt_name': 'KODEX 차이나CSI300',
   'pdno': '283580',
   'item_dvsn_name': '현금',
   'thdt_buyqty': '0',
   'thdt_sll_qty': '0',
   'hldg_qty': '20',
   'ord_psbl_qty': '20',
   'pchs_avg_pric': '10117.5000',
   'pchs_amt': '202350',
   'prpr': '10555',
   'evlu_amt': '211100',
   'evlu_pfls_amt': '8750',
   'evlu_erng_rt': '4.32419076'},
  {'cblc_dvsn_name': '사용자',
   'prdt_name': 'KODEX 200미국채혼합',
   'pdno': '284430',
   'item_dvsn_name': '현금',
   'thdt_buyqty': '0',
   'thdt_sll_qty': '0',
   'hldg_qty': '38',
   'ord_psbl_qty': '38',
   'pchs_avg_pric': '12590.2895',
   'pchs_amt': '478431',
   'prpr': '13250',
   'evlu_amt': '503500',
   'evlu_pfls_amt': '25069',
   'evlu_erng_rt': '5.23983605'},
  {

In [63]:
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

def get_balance_json(json):
    cash = json['output2'][0]['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_pfls_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)
        
        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [64]:
import pandas as pd


def get_balance_json_irp(json):
    cash = json['output2']['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_erng_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)

        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [65]:
df_0 = get_balance_json(json_0)
df_1 = get_balance_json(json_1)
df_2 = get_balance_json(json_2)
df_3 = get_balance_json(json_3)
df_4 = get_balance_json_irp(json_4)
df = pd.concat([df_0, df_1, df_2, df_3, df_4], ignore_index=True)
# df_0 = df_0.sort_values(by='이름')
df_1 = df_1.sort_values(by='이름')
df_2 = df_2.sort_values(by='이름')
df_3 = df_3.sort_values(by='이름')
df_4 = df_4.sort_values(by='이름')

total_1 = df_1["평가금액"].sum()
total_2 = df_2["평가금액"].sum()
total_3 = df_3["평가금액"].sum()
total_4 = df_4["평가금액"].sum()

In [66]:
df_1["비중"] = df_1["평가금액"] / total_1 * 100
df_1

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌,비중
2,411060,ACE KRX금현물,6935490,15110,459,6802840,14821.00,1.94,64267316-22,14.14
10,476750,ACE 미국30년국채엔화노출액티브(H),1036045,9505,109,982813,9016.63,5.41,64267316-22,2.11
0,360200,ACE 미국S&P500,5747910,18970,303,5330680,17593.00,7.82,64267316-22,11.72
6,456880,ACE 미국달러SOFR금리(합성),3015900,11170,270,2945731,10910.11,2.38,64267316-22,6.15
11,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2482245,10935,227,2698366,11887.07,-8.00,64267316-22,5.06
8,468380,KODEX iShares미국하이일드액티브,1435200,11040,130,1420250,10925.00,1.05,64267316-22,2.93
7,464540,RISE 25-03 회사채(AA-이상)액티브,2397470,51010,47,2394495,50946.70,0.12,64267316-22,4.89
9,473330,SOL 미국30년국채커버드콜(합성),10085075,10075,1001,9694000,9684.32,4.03,64267316-22,20.57
3,446720,SOL 미국배당다우존스,3318000,11060,300,3136694,10455.65,5.78,64267316-22,6.77
5,452360,SOL 미국배당다우존스(H),2240000,11200,200,2138084,10690.42,4.76,64267316-22,4.57


In [67]:
total_1

49031419

In [68]:
df_2

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
4,411060,ACE KRX금현물,1390120,15110,92,1330258,14459.33,4.50,64012548-22
1,360200,ACE 미국S&P500,1441720,18970,76,1338800,17615.79,7.68,64012548-22
7,456880,ACE 미국달러SOFR금리(합성),145210,11170,13,140018,10770.62,3.70,64012548-22
2,390390,KODEX 미국반도체MV,478275,22775,21,434120,20672.38,10.17,64012548-22
9,483230,RISE 25-06 은행채(AA+이상)액티브,1859990,50270,37,1858510,50230.00,0.07,64012548-22
8,469050,RISE 미국반도체NYSE(H),528840,14690,36,477917,13275.47,10.65,64012548-22
6,452360,SOL 미국배당다우존스(H),2128000,11200,190,1998428,10518.04,6.48,64012548-22
3,396500,TIGER Fn반도체TOP10,445500,12375,36,469560,13043.33,-5.12,64012548-22
5,449170,TIGER KOFR금리액티브(합성),3712100,106060,35,3703648,105818.51,0.22,64012548-22
0,329750,TIGER 미국달러단기채권액티브,1054850,12410,85,1013038,11918.09,4.12,64012548-22


In [69]:
total_2

13196305

In [70]:
df_3

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
1,411060,ACE KRX금현물,1949190,15110,129,1939575,15035.47,0.49,64043278-01
0,114800,KODEX 인버스,1053150,4130,255,1010475,3962.65,4.22,64043278-01
3,473330,SOL 미국30년국채커버드콜(합성),1209000,10075,120,1217010,10141.75,-0.65,64043278-01
2,449170,TIGER KOFR금리액티브(합성),3075740,106060,29,3071965,105929.83,0.12,64043278-01
4,0,현금,1894,1,1894,1894,1.00,0.00,64043278-01


In [71]:
total_3

7288974

In [72]:
df_4

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
4,411060,ACE KRX금현물,4820090,15110,319,4785871,15002.73,0.72,64012548-29
3,365780,ACE 국고채10년,271950,90650,3,261365,87121.67,4.05,64012548-29
2,360200,ACE 미국S&P500,626010,18970,33,527245,15977.12,18.73,64012548-29
6,456880,ACE 미국달러SOFR금리(합성),1452100,11170,130,1441550,11088.85,0.73,64012548-29
1,284430,KODEX 200미국채혼합,503500,13250,38,478431,12590.29,5.24,64012548-29
8,468380,KODEX iShares미국하이일드액티브,1214400,11040,110,1201700,10924.55,1.06,64012548-29
0,283580,KODEX 차이나CSI300,211100,10555,20,202350,10117.50,4.32,64012548-29
7,464540,RISE 25-03 회사채(AA-이상)액티브,8518670,51010,167,8507035,50940.33,0.14,64012548-29
9,473330,SOL 미국30년국채커버드콜(합성),3022500,10075,300,2984859,9949.53,1.26,64012548-29
5,449170,TIGER KOFR금리액티브(합성),13151440,106060,124,13110565,105730.36,0.31,64012548-29


In [73]:
total_4

33878608

In [74]:
# 그룹화하여 조건에 맞게 계산
result = df.groupby('코드').agg({
    '이름': 'first',
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
    '매입단가': 'mean',
}).reset_index()

result["비중"] = result["평가금액"] / result["평가금액"].sum() * 100
result["수익률"] = (result["평가금액"] - result["매입금액"]) / result["매입금액"] * 100
result.dropna(inplace=True)

# 결과 출력
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률
0,0,현금,120248,1.00,120248,120248,1.00,0.12,0.00
1,114800,KODEX 인버스,1053150,4130.00,255,1010475,3962.65,1.01,4.22
2,283580,KODEX 차이나CSI300,211100,10555.00,20,202350,10117.50,0.20,4.32
3,284430,KODEX 200미국채혼합,503500,13250.00,38,478431,12590.29,0.48,5.24
4,329750,TIGER 미국달러단기채권액티브,1054850,12410.00,85,1013038,11918.09,1.01,4.13
5,360200,ACE 미국S&P500,7815640,18970.00,412,7196725,17061.97,7.51,8.60
6,365780,ACE 국고채10년,271950,90650.00,3,261365,87121.67,0.26,4.05
7,390390,KODEX 미국반도체MV,478275,22775.00,21,434120,20672.38,0.46,10.17
8,396500,TIGER Fn반도체TOP10,2202750,12375.00,178,2395175,13302.00,2.12,-8.03
9,411060,ACE KRX금현물,15457530,15110.00,1023,15222184,14894.04,14.85,1.55


In [75]:
eval_total = result['평가금액'].sum()
eval_total

104094368

In [76]:
# 카테고리 분류
# 114800	KODEX 인버스
# 283580	KODEX 차이나CSI300
# 284430	KODEX 200미국채혼합
# 329750	TIGER 미국달러단기채권액티브

원화현금 = ["0", "449170",]
달러현금 = ["456880",]
국내주식 = ["396500", "114800"]
미국주식 = ["360200", "390390", "402970", "469050", "480020", "482730", "446720", "452360"]
해외주식 = ["283580", ]
미국채권 = ["284430", "329750", "473330", "476750"]
국내채권 = ["365780", "464540", "483230"]
하이일드 = ["468380", ]
대채투자 = ["411060", ]

result['카테고리'] = '기타'
result.loc[result['코드'].isin(원화현금), '카테고리'] = '원화현금'
result.loc[result['코드'].isin(달러현금), '카테고리'] = '달러현금'
result.loc[result['코드'].isin(국내주식), '카테고리'] = '국내주식'
result.loc[result['코드'].isin(미국주식), '카테고리'] = '미국주식'
result.loc[result['코드'].isin(해외주식), '카테고리'] = '해외주식'
result.loc[result['코드'].isin(미국채권), '카테고리'] = '미국채권'
result.loc[result['코드'].isin(국내채권), '카테고리'] = '국내채권'
result.loc[result['코드'].isin(하이일드), '카테고리'] = '하이일드'
result.loc[result['코드'].isin(대채투자), '카테고리'] = '대채투자'
result.sort_values(by='카테고리', inplace=True)
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리
1,114800,KODEX 인버스,1053150,4130.00,255,1010475,3962.65,1.01,4.22,국내주식
8,396500,TIGER Fn반도체TOP10,2202750,12375.00,178,2395175,13302.00,2.12,-8.03,국내주식
21,483230,RISE 25-06 은행채(AA+이상)액티브,1859990,50270.00,37,1858510,50230.00,1.79,0.08,국내채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10916140,51010.00,214,10901530,50943.52,10.49,0.13,국내채권
6,365780,ACE 국고채10년,271950,90650.00,3,261365,87121.67,0.26,4.05,국내채권
13,456880,ACE 미국달러SOFR금리(합성),4613210,11170.00,413,4527299,10923.19,4.43,1.90,달러현금
9,411060,ACE KRX금현물,15457530,15110.00,1023,15222184,14894.04,14.85,1.55,대채투자
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2482245,10935.00,227,2698366,11887.07,2.38,-8.01,미국주식
16,469050,RISE 미국반도체NYSE(H),528840,14690.00,36,477917,13275.47,0.51,10.66,미국주식
12,452360,SOL 미국배당다우존스(H),4368000,11200.00,390,4136512,10604.23,4.20,5.60,미국주식


In [77]:
# 새로운 '분류' 컬럼 추가
result['대분류'] = result['카테고리']
result.loc[result['카테고리'].str.contains('현금'), '대분류'] = '현금'
result.loc[result['카테고리'].str.contains('주식'), '대분류'] = '주식'
result.loc[result['카테고리'].str.contains('채권'), '대분류'] = '채권'
result.loc[result['카테고리'].str.contains('하이일드'), '대분류'] = '채권'
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
1,114800,KODEX 인버스,1053150,4130.00,255,1010475,3962.65,1.01,4.22,국내주식,주식
8,396500,TIGER Fn반도체TOP10,2202750,12375.00,178,2395175,13302.00,2.12,-8.03,국내주식,주식
21,483230,RISE 25-06 은행채(AA+이상)액티브,1859990,50270.00,37,1858510,50230.00,1.79,0.08,국내채권,채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10916140,51010.00,214,10901530,50943.52,10.49,0.13,국내채권,채권
6,365780,ACE 국고채10년,271950,90650.00,3,261365,87121.67,0.26,4.05,국내채권,채권
13,456880,ACE 미국달러SOFR금리(합성),4613210,11170.00,413,4527299,10923.19,4.43,1.90,달러현금,현금
9,411060,ACE KRX금현물,15457530,15110.00,1023,15222184,14894.04,14.85,1.55,대채투자,대채투자
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2482245,10935.00,227,2698366,11887.07,2.38,-8.01,미국주식,주식
16,469050,RISE 미국반도체NYSE(H),528840,14690.00,36,477917,13275.47,0.51,10.66,미국주식,주식
12,452360,SOL 미국배당다우존스(H),4368000,11200.00,390,4136512,10604.23,4.20,5.60,미국주식,주식


In [78]:
# 카테고리 별 비중 계산
대분류 = result.groupby('대분류').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = 대분류['평가금액'].sum()
대분류['비중'] = 대분류['평가금액'] / total_amount * 100
대분류["수익률"] = (대분류["평가금액"] - 대분류["매입금액"]) / 대분류["매입금액"] * 100
대분류

,대분류,평가금액,단가,수량,매입금액,비중,수익률
0,대채투자,15457530,15110.00,1023,15222184,14.85,1.55
1,주식,27325170,12698.00,2312,26668984,26.25,2.46
2,채권,32608650,31026.25,2147,32013506,31.33,1.86
3,현금,28703018,39077.00,120887,28556637,27.57,0.51


In [79]:
# 카테고리 별 비중 계산
category_group = result.groupby('카테고리').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = category_group['평가금액'].sum()
category_group['비중'] = category_group['평가금액'] / total_amount * 100
category_group["수익률"] = (category_group["평가금액"] - category_group["매입금액"]) / category_group["매입금액"] * 100
category_group

,카테고리,평가금액,단가,수량,매입금액,비중,수익률
0,국내주식,3255900,8252.50,433,3405650,3.13,-4.40
1,국내채권,13048080,63976.67,254,13021405,12.53,0.20
2,달러현금,4613210,11170.00,413,4527299,4.43,1.90
3,대채투자,15457530,15110.00,1023,15222184,14.85,1.55
4,미국주식,23858170,14274.29,1859,23060984,22.92,3.46
5,미국채권,16910970,11310.00,1653,16370151,16.25,3.30
6,원화현금,24089808,53030.50,120474,24029338,23.14,0.25
7,하이일드,2649600,11040.00,240,2621950,2.55,1.05
8,해외주식,211100,10555.00,20,202350,0.20,4.32


In [80]:
# '대분류'가 '채권'인 항목 필터링
bond_df = result[result['대분류'] == '채권']

# 총 평가금액 계산
total_bond_amount = bond_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
bond_df.loc[:, '비중'] = bond_df['평가금액'] / total_bond_amount * 100
bond_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
21,483230,RISE 25-06 은행채(AA+이상)액티브,1859990,50270.00,37,1858510,50230.00,5.70,0.08,국내채권,채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10916140,51010.00,214,10901530,50943.52,33.48,0.13,국내채권,채권
6,365780,ACE 국고채10년,271950,90650.00,3,261365,87121.67,0.83,4.05,국내채권,채권
4,329750,TIGER 미국달러단기채권액티브,1054850,12410.00,85,1013038,11918.09,3.23,4.13,미국채권,채권
3,284430,KODEX 200미국채혼합,503500,13250.00,38,478431,12590.29,1.54,5.24,미국채권,채권
17,473330,SOL 미국30년국채커버드콜(합성),14316575,10075.00,1421,13895869,9925.20,43.90,3.03,미국채권,채권
18,476750,ACE 미국30년국채엔화노출액티브(H),1036045,9505.00,109,982813,9016.63,3.18,5.42,미국채권,채권
15,468380,KODEX iShares미국하이일드액티브,2649600,11040.00,240,2621950,10924.77,8.13,1.05,하이일드,채권


In [81]:
# '대분류'가 '주식'인 항목 필터링
stock_df = result[result['대분류'] == '주식']

# 총 평가금액 계산
total_stock_amount = stock_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
stock_df.loc[:, '비중'] = stock_df['평가금액'] / total_stock_amount * 100
stock_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
1,114800,KODEX 인버스,1053150,4130.00,255,1010475,3962.65,3.85,4.22,국내주식,주식
8,396500,TIGER Fn반도체TOP10,2202750,12375.00,178,2395175,13302.00,8.06,-8.03,국내주식,주식
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2482245,10935.00,227,2698366,11887.07,9.08,-8.01,미국주식,주식
16,469050,RISE 미국반도체NYSE(H),528840,14690.00,36,477917,13275.47,1.94,10.66,미국주식,주식
12,452360,SOL 미국배당다우존스(H),4368000,11200.00,390,4136512,10604.23,15.99,5.60,미국주식,주식
10,446720,SOL 미국배당다우존스,3318000,11060.00,300,3136694,10455.65,12.14,5.78,미국주식,주식
7,390390,KODEX 미국반도체MV,478275,22775.00,21,434120,20672.38,1.75,10.17,미국주식,주식
5,360200,ACE 미국S&P500,7815640,18970.00,412,7196725,17061.97,28.60,8.60,미국주식,주식
20,482730,TIGER 미국S&P500+10%프리미엄초단기,4867170,10290.00,473,4980650,10529.92,17.81,-2.28,미국주식,주식
2,283580,KODEX 차이나CSI300,211100,10555.00,20,202350,10117.50,0.77,4.32,해외주식,주식


In [82]:
# '대분류'가 '현금'인 항목 필터링
cash_df = result[result['대분류'] == '현금']

# 총 평가금액 계산
total_cash_amount = cash_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
cash_df.loc[:, '비중'] = cash_df['평가금액'] / total_cash_amount * 100
cash_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
13,456880,ACE 미국달러SOFR금리(합성),4613210,11170.00,413,4527299,10923.19,16.07,1.90,달러현금,현금
11,449170,TIGER KOFR금리액티브(합성),23969560,106060.00,226,23909090,105872.15,83.51,0.25,원화현금,현금
0,0,현금,120248,1.00,120248,120248,1.00,0.42,0.00,원화현금,현금



## 최근 영업일

In [83]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.07.24


In [84]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20240724
